In [1]:
include(joinpath("module", "lattice.jl"));

In [2]:
using .LatticeNetwork
# using JuMP, Ipopt
using Convex, SCS

In [4]:
# 初期パラメータ設定
E = 5
Col = 2
K = Col * Col
M = 1.0
N = 1.0
Scaling = 10.0 / Col
alter_T_num = 0.5

# 格子ネットワークを生成
city_network = make_lattice(Col)

# 通勤費用パラメータ
t = 0.1
# 距離抵抗パラメータ
tau = 0.5
# 総土地供給量と平均床面積
S_total = 100
S_bar = S_total / K

# 座標データの生成
Coordinate_Data = [ 
    (city_network["node_dic"][string(i)]["x_pos"] * Scaling,
     city_network["node_dic"][string(i)]["y_pos"] * Scaling) 
    for i in 0:(K - 1)
]

# 距離行列の作成
distance_matrix = [sqrt(sum((Coordinate_Data[i] .- Coordinate_Data[j]).^2))
                   for i in 1:K, j in 1:K]

# モデルパラメータ設定
theta_firm = 1.0
theta_house = 1.0
alpha_1 = 0.4
alpha_2 = 0.4
beta_1 = 0.4
beta_2 = 0.4
# L = 1.0
# eta = 5.0
L = 0.2
eta = 1.2
p_proj = 1e-3
RW_ini = 1.0
RW_proj = 1e-3
err_short = 1e-5
err_long = 1e-3
obj_corr = 1.0
m_per = M / K
m = fill(m_per, K)
n = fill(N / (K * K), (K, K))
method = "FISTA"
dic = "Scaling=30"

K = Col * Col
T = max.(Scaling * t * alter_T_num, t * distance_matrix)
D = exp.(-tau * distance_matrix)

power_S_H_R = (1 - alpha_2) / (1 - alpha_1 - alpha_2)
power_S_H_W = alpha_2 / (1 - alpha_1 - alpha_2)
power_L_H_R = alpha_1 / (1 - alpha_1 - alpha_2)
power_L_H_W = (1 - alpha_1) / (1 - alpha_1 - alpha_2)

power_S_F_R = (1 - beta_2) / (1 - beta_1 - beta_2)
power_S_F_W = beta_2 / (1 - beta_1 -beta_2)
power_L_F_R = beta_1 / (1 - beta_1 - beta_2)
power_L_F_W = (1 - beta_1) / (1 - beta_1 - beta_2)

T_power = (1 ./ exp.(T)) .^ (1 / (1 - alpha_1 - alpha_2))
alpha_S_R = alpha_1 ^ power_S_H_R
alpha_S_W = alpha_2 ^ power_S_H_W
alpha_L_R = alpha_1 ^ power_L_H_R
alpha_L_W = alpha_2 ^ power_L_H_W

beta_S_R = beta_1 ^ power_S_F_R
beta_S_W = beta_2 ^ power_S_F_W
beta_L_R = beta_1 ^ power_L_F_R
beta_L_W = beta_2 ^ power_L_F_W

RW_S_H = zeros(K, K)
RW_L_H = zeros(K, K)

T_power_v = (1 - alpha_1 - alpha_2) * T_power
pi_beta = 1 - beta_1 - beta_2

one_K = ones(K)
one_2K = ones(2 * K)

R_ini_list = ones(K)
W_ini_list = ones(K)
RW_ini_list = ones(2 * K)

short_itr = 10
long_itr = 1000

# # 最適化実行
# RW_opt, obj_opt = solve_optimization(K, E, S_bar, m0, n0, T_power_v,
#                                      beta_L_R, beta_S_W, power_L_F_R, power_S_F_W, pi_beta,
#                                      alpha_L_R, alpha_S_W, power_L_H_R, power_S_H_W)

# println("Optimal RW: ", RW_opt)
# println("Optimal objective value: ", obj_opt)

# # パラメータの型変換
# m = Vector{Real}(m0)  # Real型に変換
# n = Matrix{Real}(n0)  # Real型に変換
# T_power_v = Matrix{Real}(T_power) * (1 - alpha_1 - alpha_2); # Real型に変換

1000

In [9]:
# 変数の定義
R = Variable(K)
W = Variable(K)

# 目的関数の定義 (ベクトル化された演算)
r_v = alpha_L_R * R.^(-power_L_H_R)
w_v = alpha_S_W * W.^(-power_S_H_W)

vn = sum((E .* W .+ T_power_v .* r_v .* w_v) .* n) # vnの計算を変更

pi_noex_result = pi_beta * (beta_L_R * R.^(-power_L_F_R)) .* (beta_S_W * W.^(-power_S_F_W))

F_value = vn + sum(pi_noex_result .* m) + S_bar * sum(R)

# 問題の定義 (制約を追加)
problem = minimize(F_value, constraints=[R >= 0, W >= 0])

# 最適化
solve!(problem, SCS.Optimizer)

# 結果の表示
println("Optimal solution found:")
println("R: ", R.value)
println("W: ", W.value)
println("Objective value: ", problem.optval)

LoadError: MethodError: no method matching ^(::Convex.IndexAtom, ::Float64)

[0mClosest candidates are:
[0m  ^([91m::ForwardDiff.Dual{Tx}[39m, ::AbstractFloat) where Tx
[0m[90m   @[39m [36mForwardDiff[39m [90mC:\Users\kimura\.julia\packages\ForwardDiff\UBbGT\src\[39m[90m[4mdual.jl:148[24m[39m
[0m  ^([91m::Irrational{:ℯ}[39m, ::Number)
[0m[90m   @[39m [90mBase[39m [90m[4mmathconstants.jl:119[24m[39m
[0m  ^([91m::Complex{T}[39m, ::S) where {T<:Real, S<:Real}
[0m[90m   @[39m [90mBase[39m [90m[4mcomplex.jl:876[24m[39m
[0m  ...


In [5]:
# 修正された z_sd_jump 関数
function z_sd_jump(r_vec::Vector{T}, w_vec::Vector{T}, k::Int64, e::T, s_bar::T, m::Vector{T}, n::Matrix{T}, t_power_v::Matrix{T},
    beta_l_r::T, beta_s_w::T, power_l_f_r::T, power_s_f_w::T, pi_beta::T,
    alpha_l_r::T, alpha_s_w::T, power_l_h_r::T, power_s_h_w::T) where {T<:Real}

    r_v = [alpha_l_r * r_vec[i]^(-power_l_h_r) for i in eachindex(r_vec)]
    w_v = [alpha_s_w * w_vec[i]^(-power_s_h_w) for i in eachindex(w_vec)]

    vn = sum((e * w_vec[j] + t_power_v[i, j] * r_v[i] * w_v[j]) * n[i, j] for i in 1:k, j in 1:k)

    pi_noex_result = [pi_beta * (beta_l_r * r_vec[i]^(-power_l_f_r)) * (beta_s_w * w_vec[i]^(-power_s_f_w)) for i in eachindex(r_vec)]

    f_value = vn + sum(pi_noex_result .* m) + s_bar * sum(r_vec)

    return f_value
end


# モデルの定義
model = Model(Ipopt.Optimizer)

# z_sd_jump関数を登録  <-- 修正された関数を登録
register(model, :z_sd_jump, 18, z_sd_jump, autodiff=true)

# 変数の定義 (RとW)
@variable(model, R[1:K] >= 0)
@variable(model, W[1:K] >= 0)

# 目的関数の設定 (非線形)
@NLobjective(model, Min, :z_sd_jump(R, W, K, E, S_bar, m, n, T_power_v,
    beta_L_R, beta_S_W, power_L_F_R, power_S_F_W, pi_beta,
    alpha_L_R, alpha_S_W, power_L_H_R, power_S_H_W))

# 最適化
optimize!(model)

# 結果の表示 (前回と同じ)
if termination_status(model) == OPTIMAL
    println("Optimal solution found:")
    println("R: ", value.(R))
    println("W: ", value.(W))
    println("Objective value: ", objective_value(model))
else
    println("Optimization did not converge or found no solution.")
    println(termination_status(model))
end

LoadError: Unable to register the function :z_sd_jump.

Common reasons for this include:

 * The function takes `f(x::Vector)` as input, instead of the splatted
   `f(x...)`.
 * The function assumes `Float64` will be passed as input, it must work for any
   generic `Real` type.
 * The function allocates temporary storage using `zeros(3)` or similar. This
   defaults to `Float64`, so use `zeros(T, 3)` instead.

As examples, instead of:
```julia
my_function(x::Vector) = sum(x.^2)
```
use:
```julia
my_function(x::T...) where {T<:Real} = sum(x[i]^2 for i in 1:length(x))
```

Instead of:
```julia
function my_function(x::Float64...)
    y = zeros(length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```
use:
```julia
function my_function(x::T...) where {T<:Real}
    y = zeros(T, length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```

Review the stacktrace below for more information, but it can often be hard to
understand 

In [13]:
# pi_noex の定義（変数に対応）
function pi_noex(R, W, beta_L_R, beta_S_W, power_L_F_R, power_S_F_W, pi_beta)
    return [pi_beta * (beta_L_R * R[i]^(-power_L_F_R)) * (beta_S_W * W[i]^(-power_S_F_W)) for i in eachindex(R)]
end

# JuMP の非線形関数として登録可能な形に変更
function Z_SD(RW_vec::Vararg{T}, K::T, E::T, S_bar::T, m::Matrix{T}, n::Matrix{T}, T_power_v::Matrix{T},
              beta_L_R::T, beta_S_W::T, power_L_F_R::T, power_S_F_W::T, pi_beta::T,
              alpha_L_R::T, alpha_S_W::T, power_L_H_R::T, power_S_H_W::T) where {T<:Real}

    RW_vec_array = collect(RW_vec)  # タプルを配列に変換
    R = RW_vec_array[1:K]
    W = RW_vec_array[K+1:2K]

    R_v = [alpha_L_R * R[i]^(-power_L_H_R) for i in eachindex(R)]
    W_v = [alpha_S_W * W[i]^(-power_S_H_W) for i in eachindex(W)]

    vn = sum((E * W[j] + T_power_v[i, j] * R_v[i] * W_v[j]) * n[i, j] for i in 1:K, j in 1:K)

    pi_noex_result = pi_noex(R, W, beta_L_R, beta_S_W, power_L_F_R, power_S_F_W, pi_beta)

    F_value = vn + sum(pi_noex_result .* m) + S_bar * sum(R)

    return F_value
end

# 最適化関数
function solve_optimization(K, E, S_bar, m, n, T_power_v,
                            beta_L_R, beta_S_W, power_L_F_R, power_S_F_W, pi_beta,
                            alpha_L_R, alpha_S_W, power_L_H_R, power_S_H_W)

    model = Model(Ipopt.Optimizer)  # Ipopt を使用（非線形最適化）

    # Z_SD を JuMP に登録
    register(model, :Z_SD, 15, (x...) -> Z_SD(x...), autodiff = true)

    # 変数 RW（R と W を 1 つのベクトルにまとめる）
    @variable(model, RW[1:2K] >= 0)

    # 目的関数を非線形式として設定
    @NLobjective(model, Min, Z_SD(RW..., K, E, S_bar, m, n, T_power_v,
                                  beta_L_R, beta_S_W, power_L_F_R, power_S_F_W, pi_beta,
                                  alpha_L_R, alpha_S_W, power_L_H_R, power_S_H_W))

    optimize!(model)

    return value.(RW), objective_value(model)
end


LoadError: ArgumentError: Vararg on non-final argument in method definition for Z_SD at In[13]:7